# Analyzers Basic Tutorial

This Jupyter notebook will give a basic tutorial on how to use PyDeequ's Analyzers module.

In [1]:
from pyspark.sql import SparkSession, Row, DataFrame
import json
import pandas as pd
import sagemaker_pyspark

import pydeequ

classpath = ":".join(sagemaker_pyspark.classpath_jars())

spark = (SparkSession
    .builder
    .config("spark.driver.extraClassPath", classpath)
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

### We will be using the Amazon Product Reviews dataset -- specifically the Electronics subset. 

In [2]:
df = spark.read.parquet("s3a://amazon-reviews-pds/parquet/product_category=Electronics/")

df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- year: integer (nullable = true)



In [3]:
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("review_id")) \
                    .addAnalyzer(ApproxCountDistinct("review_id")) \
                    .addAnalyzer(Mean("star_rating")) \
                    .addAnalyzer(Compliance("top star_rating", "star_rating >= 4.0")) \
                    .addAnalyzer(Correlation("total_votes", "star_rating")) \
                    .addAnalyzer(Correlation("total_votes", "helpful_votes")) \
                    .run()
                    
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-----------+--------------------+-------------------+--------------------+
|     entity|            instance|               name|               value|
+-----------+--------------------+-------------------+--------------------+
|     Column|           review_id|       Completeness|                 1.0|
|     Column|           review_id|ApproxCountDistinct|           3010972.0|
|Mutlicolumn|total_votes,star_...|        Correlation|-0.03451097996538765|
|    Dataset|                   *|               Size|           3120938.0|
|     Column|         star_rating|               Mean|   4.036143941340712|
|     Column|     top star_rating|         Compliance|  0.7494070692849394|
|Mutlicolumn|total_votes,helpf...|        Correlation|  0.9936463809903863|
+-----------+--------------------+-------------------+--------------------+



In [5]:
analysisResult_pandas_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult, pandas=True)
analysisResult_pandas_df

,entity,instance,name,value
0,Column,review_id,Completeness,1.000000e+00
1,Column,review_id,ApproxCountDistinct,3.010972e+06
2,Mutlicolumn,"total_votes,star_rating",Correlation,-3.451098e-02
3,Dataset,*,Size,3.120938e+06
4,Column,star_rating,Mean,4.036144e+00
5,Column,top star_rating,Compliance,7.494071e-01
6,Mutlicolumn,"total_votes,helpful_votes",Correlation,9.936464e-01


### For more info ... look at full list of analyzers in `docs/analyzers.md` 